# Tire Quality Classification

**Over time, rubber deteriorates - can we detect this by a simple image?**

- Objective: Develop a model to help predict the quality of car tires.
- Dataset: 1854 images available.

### Transfer Learning

- A machine learning technique where a model trained on one task is reused or adapted for a new, related task.
- Reduce training time and computational ressources.
- Achieves better perfomances, especially when data is limited in the new task.

**How it works?**

- We start with a pre-trained model (tipically trained on a large dataset).
- We then adjust the model to optimize it for the new task.

**The model: ResNet**

- Residual Network is a deep neural network architecture.
- Trained on the ImageNet dataset.
- **Key Concept**: Allow layers to "skip" connections, helping compating the vanishing gradient problem, enabling deeper networks.

In [1]:
import torch
from torch import nn

In [2]:
if torch.cuda.is_available:
    device = torch.device("cuda")

False